In [30]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from tqdm.auto import tqdm

data_dir = '/home/pratzohol/google-drive/work-stuff/harmful-meme-detection/datasets/fb-data'
img_dir = os.path.join(data_dir, 'img')

In [31]:
splits = ['train', 'val', 'test']

df = []
for split in splits:
    if split == 'val':
        split = 'dev'

    file_path = os.path.join(data_dir, f'{split}.jsonl')
    split_df = pd.read_json(file_path, lines=True)
    split_df['split'] = split
    df.append(split_df)


In [32]:
df = pd.concat(df, axis=0, ignore_index=True)
df['id'] = df['img'].str.split('/').str[1].str.split('.').str[0]
df.index = df['id']
df.index.name = None
df.loc[df["split"] == 'dev', "split"] = "val"
print(df.shape)
df.head()

(10000, 5)


,id,img,label,text,split
42953,42953,img/42953.png,0.0,its their character not their color that matters,train
23058,23058,img/23058.png,0.0,don't be afraid to love again everyone is not ...,train
13894,13894,img/13894.png,0.0,putting bows on your pet,train
37408,37408,img/37408.png,0.0,i love everything and everybody! except for sq...,train
82403,82403,img/82403.png,0.0,"everybody loves chocolate chip cookies, even h...",train


In [33]:
df['split'].value_counts()

split
train    8500
test     1000
val       500
Name: count, dtype: int64

In [34]:
df['label'].value_counts(normalize=False)

label
0.0    5700
1.0    3300
Name: count, dtype: int64

In [35]:
# df["text"].to_csv("english.txt", index=False, header=False)

In [36]:
import cv2
import pytesseract

def preprocess_image(im):
    """Summary

    Args:
        im (np.array): Image in BGR format after using cv2.imread(<filePath>)

    Returns:
        np.array :
    """
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    im = cv2.bilateralFilter(im, 9, 55, 60)
    _, im = cv2.threshold(im, 235, 255, cv2.THRESH_BINARY_INV)
    return im


def extract_text_from_meme_eng(file_path):
    im = cv2.imread(file_path)
    im = preprocess_image(im)

    tess_config = r'-l eng --tessdata-dir /usr/share/tesseract-ocr/tessdata_best --oem 1 --psm 11'
    txt = pytesseract.image_to_string(im, config=tess_config)
    txt = txt.replace('\n', ' ').strip()

    return txt


def extract_text_from_meme_all(file_path):
    im = cv2.imread(file_path)
    im = preprocess_image(im)

    tess_config = r'-l eng+chi_sim+chi_tra+tam+msa --tessdata-dir /usr/share/tesseract-ocr/tessdata_best --oem 1 --psm 11'
    txt = pytesseract.image_to_string(im, config=tess_config)
    txt = txt.replace('\n', ' ').strip()

    return txt

In [37]:
file_paths = list(df["img"])
file_paths = [f"{data_dir}/{fp}" for fp in file_paths]

dir = "/home/pratzohol/google-drive/work-stuff/harmful-meme-detection/datasets/fb-data/ocr/"
all_dir = dir + "all.txt"
eng_dir = dir + "eng.txt"


In [41]:
# from tqdm import tqdm

# with open(all_dir, "w") as f:
#     for fp in tqdm(file_paths):
#         txt_all = extract_text_from_meme_all(fp)
#         f.write(txt_all)
#         f.write("\n")

100%|██████████| 10000/10000 [3:48:59<00:00,  1.37s/it] 


In [42]:
# with open(eng_dir, "w") as f:
#     for fp in tqdm(file_paths):
#         txt_eng = extract_text_from_meme_eng(fp)
#         f.write(txt_eng)
#         f.write("\n")

100%|██████████| 10000/10000 [37:15<00:00,  4.47it/s]
